#### Faiss
Facebook AI Similarity Search (Faiss) is a library for efficient similarity search and clustering of dense vectors. It contains algorithms that search in sets of vectors of any size, up to ones that possibly do not fit in RAM. It also contains supporting code for evaluation and parameter tuning.

In [1]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_ollama import OllamaEmbeddings
from langchain_text_splitters import CharacterTextSplitter

loader=TextLoader("speech.txt")
documents=loader.load()
text_splitter=CharacterTextSplitter(chunk_size=1000,chunk_overlap=30)
docs=text_splitter.split_documents(documents)


In [2]:
docs
type(docs[0])

langchain_core.documents.base.Document

In [3]:
# zsh `ollama run gemma2:2b`
embeddings=(
    OllamaEmbeddings(model="llama2")  ##by default it ues llama2
)
db=FAISS.from_documents(docs,embeddings)
db

In [4]:
### querying 
query="How does the speaker describe the desired outcome of the war?"
docs=db.similarity_search(query)
docs[0].page_content


'It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our hearts—for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'

#### As a Retriever
We can also convert the vectorstore into a Retriever class. This allows us to easily use it in other LangChain methods, which largely work with retrievers

In [5]:
retriever=db.as_retriever()
docs=retriever.invoke(query)
docs[0].page_content

'It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our hearts—for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'

#### Similarity Search with score
There are some FAISS specific methods. One of them is similarity_search_with_score, which allows you to return not only the documents but also the distance score of the query to them. The returned distance score is L2 distance. Therefore, a lower score is better.

In [6]:
docs_and_score=db.similarity_search_with_score(query)
docs_and_score

[(Document(id='9b5d1e1d-683c-4479-9a5f-741bed4c0a75', metadata={'source': 'speech.txt'}, page_content='It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our hearts—for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'),
  1.0497818),
 (Document(id='854f3f84-7bf0-4154-bcd2-344b4c14c8f6', metadata={'source': 'speech.txt'}, p

In [7]:
embedding_vector=embeddings.embed_query(query)
embedding_vector

[0.02138579,
 -0.029378856,
 0.010500348,
 0.014126486,
 -0.0066914037,
 0.003275847,
 0.010389574,
 0.002120735,
 0.00483143,
 -0.021922916,
 0.009232413,
 -0.018629687,
 -0.006755551,
 0.00435958,
 -0.0087896865,
 -0.015517958,
 -0.009227761,
 -0.0037680534,
 0.015234566,
 5.6369143e-05,
 -0.0072492072,
 -0.0022052785,
 0.018113011,
 -0.017053938,
 0.0055572176,
 -0.005965714,
 0.00014562433,
 -0.0060751396,
 0.0064742863,
 -0.0074152094,
 0.0138289165,
 -0.028270403,
 -0.01068884,
 0.02997257,
 0.012605421,
 -0.041671313,
 -0.017774595,
 0.0020414104,
 -0.016526084,
 -0.013608992,
 -0.0046703373,
 -0.01699188,
 0.0148015255,
 0.006725798,
 0.0016254785,
 0.003917958,
 -0.011140288,
 0.009913858,
 0.0078384895,
 -0.018356381,
 -0.028043572,
 0.010263559,
 0.026895454,
 -0.0052621095,
 0.027641155,
 -0.0002701196,
 -0.010591818,
 0.007766586,
 -0.007842918,
 -0.006866063,
 -0.0015923711,
 -0.003538055,
 0.008008971,
 0.0013378431,
 -0.011519251,
 -0.0053539,
 -0.0050088605,
 -0.002204

In [8]:
docs_score=db.similarity_search_by_vector(embedding_vector)
docs_score

[Document(id='9b5d1e1d-683c-4479-9a5f-741bed4c0a75', metadata={'source': 'speech.txt'}, page_content='It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our hearts—for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'),
 Document(id='854f3f84-7bf0-4154-bcd2-344b4c14c8f6', metadata={'source': 'speech.txt'}, page_content='…\n

In [9]:
### Saving And Loading
db.save_local("faiss_index")

In [10]:
new_db=FAISS.load_local("faiss_index",embeddings,allow_dangerous_deserialization=True)
docs=new_db.similarity_search(query)

In [11]:
docs

[Document(id='9b5d1e1d-683c-4479-9a5f-741bed4c0a75', metadata={'source': 'speech.txt'}, page_content='It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our hearts—for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'),
 Document(id='854f3f84-7bf0-4154-bcd2-344b4c14c8f6', metadata={'source': 'speech.txt'}, page_content='…\n